""" 

**Extract slices of the volume that contain the pancreas/pancreatic cyst.**

""" 

In [12]:
import os
import SimpleITK as sitk
import pathlib as pathlib
import numpy as np
import pandas as pd


def make_directory(path):
    try:
        os.mkdir(path)
        
    except FileExistsError:
        print("-----FAILED: "+path+ " Already exists")
    except OSError:
        print("Failed to create directory in: " +path)
    else:
        print("Directory created in: " +path)
        return None
    try:
        os.makedirs(path)
    except FileExistsError:
        print("-----FAILED: "+path+ " Already exists")
    except OSError:
        print("Failed to create directory in: " +path)
    else:
        print("Directory created in: " +path)
        return None

def normalize(image):
    #input sitk.image object
    mr_arr = sitk.GetArrayFromImage(image)
    max_val = mr_arr.max()
    mr_arr = mr_arr / max_val
    return mr_arr

def make_directory(path):
    try:
        os.mkdir(path)
    except FileExistsError:
        print("-----FAILED: "+path+ " Already exists")
    except OSError:
        print("Failed to create director in: " +path)
    else:
        print("Directory created in: " +path)

def sliceExtractor(path_to_abd_ct,panc_slice_index,outputDir_path):
    abdominal_ct_Data = pathlib.Path(path_to_abd_ct)# path to resampled .mhd files 
    print(abdominal_ct_Data)
    patientFiles = [file for file in abdominal_ct_Data.iterdir()] 
    
    for volume in patientFiles:
        if '.mhd' in str(volume):
        
            patientFile = (str(volume.stem)+"_extracted_slices") 
            slice_index_key = str(volume.stem[0:4]) 
            image = sitk.ReadImage(str(volume), imageIO = "MetaImageIO") #read nrrd
            normalized_img_arr = normalize(image)  #normalize it 
            normalized_Img = sitk.GetImageFromArray(normalized_img_arr) #get new sitk.image from normalized numpy arr
            

            slice_index = panc_slice_index[slice_index_key]
            #sitk.ResampleFilter adds 256 slice at the end of the resampled image. 
            #Remove last item from list. 
            if slice_index[len(slice_index)-1]-slice_index[0] != 1:
                slice_index = slice_index[0:(len(slice_index)-1)]
            new_image_arr = normalized_img_arr[slice_index[0]:slice_index[len(slice_index)-1]]
            new_image = sitk.GetImageFromArray(new_image_arr)
            sitk.WriteImage(new_image,outputDir_path+"/"+patientFile+".mhd")


def pancreasIndexExtractor(path_seg_panc):
    patient_slice_index = {}
    segmented_panc_Data = pathlib.Path(path_seg_panc)# path to resampled .mhd files 
    patientFiles = [file for file in segmented_panc_Data.iterdir()] 
    indx=0 
    for volume in patientFiles:
        if '.mhd' in str(volume):
            patientID = str(volume.stem[0:4]) 
            image = sitk.ReadImage(str(volume), imageIO = "MetaImageIO")
            ct_arr = sitk.GetArrayFromImage(image)
            slice_list = []
            for i in range(len(ct_arr)):
                ct_slice = ct_arr[i,:,:]
                max_val = ct_slice.max()
                if max_val != -1000:
                    slice_list.append(i)
        
            patient_slice_index[patientID] = slice_list
    return patient_slice_index

def main(): 


    path_abd_ct="C:/Users/salch/pancreas/data/resampled_whole_image"
    path_seg_panc ='C:/Users/salch/pancreas/data/resampled_seg_panc'
    outputDir_path = 'c:/Users/salch/pancreas/data/slice_extracted_whole_image'
    make_directory(outputDir_path)
    
    panc_slice_index = pancreasIndexExtractor(path_seg_panc)
    sliceExtractor(path_abd_ct,panc_slice_index,outputDir_path)

main()

# path_abd_ct="C:/Users/salch/pancreas/data/resampled_whole_image"
path_seg_panc ='C:/Users/salch/pancreas/data/resampled_seg_panc'
panc_slice_index = pancreasIndexExtractor(path_seg_panc)
#sliceExtractor(path_abd_ct,panc_slice_index,outputDir_path)





In [16]:
example_patient = panc_slice_index['1275']
print("CT slices containing segmented pancreas in Patient 1275: \n" + str(example_patient))



CT slices containing segmented pancreas in Patient 1275: 
[35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 89]
